This notebook sets up the Agentic system.
The simplified flow of the system is as follows:
User gives input question as a prompt -> Preprocess the question via prompt tuning -> do a semantic search query over the summary database  -> 

(branch 1): is the answer in a summary (or spread over multiple)? -> use the summary/summaries to generate summary answer -> output answer to user

(branch 2): is the answer not in a summary? -> semantic search query the entire dataset for the answer -> retrieve the relevant chunks -> generate summary of the relevant results -> output answer to user

In [ ]:
# rizzbot_agentic.py with logging

import os
import numpy as np
from typing import List, Dict, Optional
from langchain.schema.runnable import RunnableLambda, RunnableBranch
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.agents import AgentExecutor, Tool, initialize_agent, AgentType
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone
from langsmith import Client
from langchain.retrievers.multi_query import MultiQueryRetriever


class Rizzbot:
    def __init__(self):
        print("[INIT] Starting Rizzbot initialization...")
        _ = self._load_env()
        self.similarity_threshold = 0.65
        self.top_k = 3

        os.environ["LANGCHAIN_TRACING_V2"] = "true"
        os.environ["LANGCHAIN_PROJECT"] = "rizzbot"
        print("[ENV] Environment variables set.")

        self.client = Client()

        self.main_llm = ChatOpenAI(model="gpt-4o", temperature=0.25)
        print("[LLM] Main LLM (gpt-4o) initialized.")

        self.expand_llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.6)
        print("[LLM] Expansion LLM (gpt-3.5-turbo) initialized.")

        self.embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
        print("[Embeddings] OpenAI embeddings initialized.")

        self.pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
        print("[Pinecone] Pinecone client initialized.")

        self.summaries_vectorstore = PineconeVectorStore(
            index_name="rizzbot-summaries-full-text",
            embedding=self.embeddings,
            text_key="full_text"
        )
        print("[VectorStore] Summaries vector store initialized.")

        self.full_vectorstore = PineconeVectorStore(
            index_name="rizzbot", embedding=self.embeddings, text_key="full_text"
        )
        print("[VectorStore] Full vector store initialized.")

        self.no_answer_response = "Sorry bro, I couldn't find enough info to answer that confidently."

        self.base_prompt_template = ChatPromptTemplate.from_template("""
        You are a charisma and personal development expert helping someone improve their social skills.

        Context: {content}
        Question: {question}

        Instructions:
        1. Analyze the question and context. Check the vectorstores for an answer. If the answer can not be found in the vectorstore, answer: "Sorry bro, I couldn't find enough info in my database to answer that confidently.""
        2. If the question is not clear, ask for clarification.
        3. If the question is clear, provide actionable, specific advice based on the context.
        4. Use examples when possible
        5. Keep the tone encouraging and supportive
        6. If information is insufficient, explain what you'd need to give a better answer

        Response:
        """)


        self._build_agent_chain()
        print("[INIT] Rizzbot initialized and ready.")

    def _load_env(self):
        from dotenv import load_dotenv, find_dotenv
        print("[ENV] Loading environment variables from .env file...")
        return load_dotenv(find_dotenv())

    def _embed_question(self, question: str) -> List[float]:
        print(f"[Embed] Embedding question: {question}")
        result = self.embeddings.embed_query(question)
        print(f"[Embed] Embedding result length: {len(result)}")
        return result

    def _cosine_similarity(self, vec1, vec2):
        vec1, vec2 = np.array(vec1), np.array(vec2)
        return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

    def _filter_by_similarity(self, query_embedding, docs, threshold):
        filtered = []

        for doc in docs:
            try:
                doc_embedding = self.embeddings.embed_query(doc.page_content)
                sim = self._cosine_similarity(query_embedding, doc_embedding)
                print(f"[Similarity] Score: {sim:.4f} | Text: {doc.page_content[:80]}...")

                if sim >= threshold:
                    filtered.append(doc)
            except Exception as e:
                print(f"[Similarity] Failed to embed doc: {e}")

        return filtered

    def _hybrid_query_search(self, question: str) -> List[str]:
        print(f"[Search:Hybrid] Embedding question...")
        question_embedding = self._embed_question(question)
        combined_results = []

        for label, vectorstore in [("summaries", self.summaries_vectorstore), ("full", self.full_vectorstore)]:
            print(f"[Search:Hybrid] Trying {label} vectorstore...")

            try:
                retriever = MultiQueryRetriever.from_llm(
                    retriever=vectorstore.as_retriever(search_kwargs={"k": self.top_k}),
                    llm=self.expand_llm
                )
                docs = retriever.invoke(question)
                filtered = self._filter_by_similarity(question_embedding, docs, self.similarity_threshold)
                print(f"[Search:Hybrid] {len(filtered)} docs passed threshold in {label}.")
                combined_results.extend([doc.page_content for doc in filtered])
            except Exception as e:
                print(f"[Search:Hybrid] Retrieval failed for {label}: {e}")

        return combined_results

    def _build_agent_chain(self):
        print("[Chain] Building agent chain...")

        def hybrid_search(q):
            results = self._hybrid_query_search(q)
            if results:
                return "\n\n".join(results)  # merge the filtered context into one string
            else:
                return self.no_answer_response
    
        self.agent_chain = (
            {
                "question": lambda q: q,
                "content": hybrid_search,
            }
            | self.base_prompt_template
            | self.main_llm
            | StrOutputParser()
        )

        print("[Chain] Agent chain constructed.")

    def answer_question(self, question: str) -> str:
        print(f"[Answer] Received question: {question}")
        try:
            context = self._hybrid_query_search(question)
            if not context:
                print("[Answer] No relevant documents found. Returning fallback response.")
                return self.no_answer_response

            print("[Answer] Relevant context found. Generating response with LLM...")
            answer = self.agent_chain.invoke(question)
            print(f"[Answer] Answer generated successfully.")
            return answer
        except Exception as e:
            print(f"[Answer] Agentic pipeline failed: {e}")
            return self.no_answer_response


if __name__ == "__main__":
    print("[Test] Starting test run for Rizzbot...\n")
    
    bot = Rizzbot()
    sample_question = "What is the capital of Russia?"

    print(f"\n[Test] Asking: {sample_question}\n")
    answer = bot.answer_question(sample_question)

    print("\n[Test] Final Answer:")
    print(answer)


[Test] Starting test run for Rizzbot...

[INIT] Starting Rizzbot initialization...
[ENV] Loading environment variables from .env file...
[ENV] Environment variables set.
[LLM] Main LLM (gpt-4o) initialized.
[LLM] Expansion LLM (gpt-3.5-turbo) initialized.
[Embeddings] OpenAI embeddings initialized.
[Pinecone] Pinecone client initialized.
[VectorStore] Summaries vector store initialized.
[VectorStore] Full vector store initialized.
[Chain] Building agent chain...
[Chain] Agent chain constructed.
[INIT] Rizzbot initialized and ready.

[Test] Asking: What is the capital of Russia?

[Answer] Received question: What is the capital of Russia?
[Search:Hybrid] Embedding question...
[Embed] Embedding question: What is the capital of Russia?
[Embed] Embedding result length: 1536
[Search:Hybrid] Trying summaries vectorstore...
[Similarity] Score: 0.0912 | Text: Topic ID: 16
Run Name: 1000word_summaries_20250702_132026
Target Words: 1000
Tim...
[Similarity] Score: 0.0813 | Text: Topic ID: 33
Run N

In [3]:
# test_rizzbot.py

from rizzbot_agentic import Rizzbot

if __name__ == "__main__":
    print("[Test] Starting test run for Rizzbot...\n")
    
    bot = Rizzbot()
    sample_question = "What makes the character Harvey Specter from Suits charismatic?"

    print(f"\n[Test] Asking: {sample_question}\n")
    answer = bot.answer_question(sample_question)

    print("\n[Test] Final Answer:")
    print(answer)


[Test] Starting test run for Rizzbot...


[Test] Asking: What makes the character Harvey Specter from Suits charismatic?


[Test] Final Answer:
Harvey Specter from "Suits" is a quintessential example of charisma, combining confidence, strategic thinking, and a commanding presence. Here’s how you can incorporate some of his charismatic traits into your own social interactions:

1. **Master the Art of Framing**:
   - **Actionable Advice**: Control the narrative in conversations by setting the context. This means being clear about your intentions and guiding discussions towards your strengths.
   - **Example**: If you're in a meeting, start by outlining the key points you want to address. This sets the stage for you to lead the conversation and ensures that your perspective is heard.

2. **Exude Confidence**:
   - **Actionable Advice**: Confidence is not just about what you say, but how you say it. Practice speaking slowly and clearly, maintaining eye contact, and using deliberate gesture

In [2]:
from rizzbot_agentic import Rizzbot

bot = Rizzbot()

# Agentic chain route
response = bot.answer_question("What makes David Dobrik popular?")
print("Chain Output:\n", response)

response2 = bot.answer_question("What are the masculine archetypes, and how do these relate to charisma?")
print("Chain Output:\n", response2)


Chain Output:
 David Dobrik's popularity can be attributed to several key factors that you can incorporate into your own social interactions to enhance your charisma and personal appeal. Here’s how you can apply these principles:

1. **Embrace Humor**:
   - **Actionable Advice**: Incorporate humor into your interactions. Don't be afraid to share funny stories or make light-hearted jokes, even if they are a bit silly. Laughter creates a shared experience that strengthens bonds.
   - **Example**: If you’re in a group setting, share a humorous anecdote from your day. For instance, if you tripped over your shoelaces, you might say, “I was practicing my dance moves today and the floor decided to join in.”

2. **Be Authentic**:
   - **Actionable Advice**: Present your true self rather than a polished version. Authenticity is relatable and endearing. Engage with others from a place of joy and confidence.
   - **Example**: If you’re passionate about a hobby, talk about it with enthusiasm. Let 

In [10]:
from rizzbot_agentic_v2 import Rizzbot


def test_rizzbot():
    print("Starting Rizzbot test...")
    
    # Initialize bot (you'll see all the initialization logs)
    bot = Rizzbot()
    
    # Check system stats first
    print("\n System Stats:")
    stats = bot.get_stats()
    for key, value in stats.items():
        print(f"  {key}: {value}")
    
    print("\n" + "="*80)
    print("TEST 1: Game of Thrones Characters")
    print("="*80)
    
    response1 = bot.answer_question("Who are the most charismatic characters in Game of Thrones and why?")
    print("\nChain Output:")
    print(response1)
    
    print("\n" + "="*80)
    print("TEST 2: Masculine Archetypes")
    print("="*80)
    
    response2 = bot.answer_question("What are the masculine archetypes, and how do these relate to charisma?")
    print("\n Chain Output:")
    print(response2)
    
    print("\n Test completed!")

if __name__ == "__main__":
    test_rizzbot()


2025-07-02 16:56:57,307 - INFO - Initializing Rizzbot...
2025-07-02 16:56:57,309 - INFO - Loading environment variables...
2025-07-02 16:56:57,314 - INFO - Setting up LLM models...


Starting Rizzbot test...


2025-07-02 16:56:58,037 - INFO - Connecting to Pinecone vector stores...
2025-07-02 16:56:58,040 - INFO - Setting up prompt templates...
2025-07-02 16:56:58,041 - INFO - Building agent chain...
2025-07-02 16:56:58,041 - INFO - Building agent execution chain...


TypeError: 'function' object is not a mapping